## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-23-26-53 +0000,nyt,Mexico Sentences 10 Men to 140 Years Each in P...,https://www.nytimes.com/2025/07/09/world/ameri...
1,2025-07-09-23-15-46 +0000,nyt,Trump Tariff Letters: See the Latest Rates and...,https://www.nytimes.com/2025/07/07/business/tr...
2,2025-07-09-23-03-27 +0000,bbc,"Package holidays to Spain, Cyprus and Turkey s...",https://www.bbc.com/news/articles/clyndp097gro
3,2025-07-09-23-01-22 +0000,bbc,'I can't drink the water' - life next to a US ...,https://www.bbc.com/news/articles/cy8gy7lv448o
4,2025-07-09-23-01-03 +0000,bbc,India and China strive to reset ties but with ...,https://www.bbc.com/news/articles/cly25ndp8kwo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,15
0,mexico,5
37,tariffs,5
89,texas,4
183,deal,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
5,2025-07-09-22-57-20 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,34
8,2025-07-09-22-14-22 +0000,nyt,Trump Discusses Economic Investment With Afric...,https://www.nytimes.com/2025/07/09/us/politics...,29
7,2025-07-09-22-32-07 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo,29
10,2025-07-09-21-41-44 +0000,nyt,Trump Tariffs Aim to Settle Scores With Countr...,https://www.nytimes.com/2025/07/09/business/ec...,29
37,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo,29


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
5,34,2025-07-09-22-57-20 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
55,18,2025-07-09-00-11-42 +0000,bbc,Moment house is swept away in New Mexico flash...,https://www.bbc.com/news/videos/cren2zq1y3qo
6,17,2025-07-09-22-33-01 +0000,cbc,What we know (so far) about the plot to seize ...,https://www.cbc.ca/news/politics/rcmp-militia-...
13,16,2025-07-09-20-36-58 +0000,bbc,'Huge wall of water': Texas man stood on meter...,https://www.bbc.com/news/articles/c3358pz41d6o
27,14,2025-07-09-16-06-35 +0000,bbc,EU hopes to agree US tariff deal 'in coming days',https://www.bbc.com/news/articles/cev0n01y8wwo
28,13,2025-07-09-15-36-01 +0000,bbc,Ancelotti gets jail term for tax evasion but w...,https://www.bbc.com/sport/football/articles/cz...
54,12,2025-07-09-00-34-08 +0000,bbc,Are India's skies safe? Air safety watchdog re...,https://www.bbc.com/news/articles/cvgnr828z0go
40,12,2025-07-09-12-19-38 +0000,bbc,"Get your act together, Post Office victims tel...",https://www.bbc.com/news/articles/crk6r2615vzo
8,11,2025-07-09-22-14-22 +0000,nyt,Trump Discusses Economic Investment With Afric...,https://www.nytimes.com/2025/07/09/us/politics...
34,10,2025-07-09-13-01-52 +0000,nyt,Philadelphia Union Reaches Deal to End Strike ...,https://www.nytimes.com/2025/07/09/world/ameri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
